In [1]:
#*----------------------------------------------------------------------------*
#* Copyright (C) 2020 ETH Zurich, Switzerland                                 *
#* SPDX-License-Identifier: Apache-2.0                                        *
#*                                                                            *
#* Licensed under the Apache License, Version 2.0 (the "License");            *
#* you may not use this file except in compliance with the License.           *
#* You may obtain a copy of the License at                                    *
#*                                                                            *
#* http://www.apache.org/licenses/LICENSE-2.0                                 *
#*                                                                            *
#* Unless required by applicable law or agreed to in writing, software        *
#* distributed under the License is distributed on an "AS IS" BASIS,          *
#* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.   *
#* See the License for the specific language governing permissions and        *
#* limitations under the License.                                             *
#*                                                                            *
#* Author:  Thorir Mar Ingolfsson                                             *
#*----------------------------------------------------------------------------*

#!/usr/bin/env python3

import numpy as np
import scipy.io as sio
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import load_model
import os
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline 
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from joblib import dump, load
import pickle
from sklearn.metrics import cohen_kappa_score
from utils.data_loading import prepare_features
def build_model(path):
    model = load_model(path)
    #model = load_model(path +'best.h5')
    for l in model.layers:
        l.trainable = False
    lr = 0.001
    model.compile(loss = 'categorical_crossentropy',optimizer=Adam(lr=lr),metrics=['accuracy'])
    return model

class Scaler( BaseEstimator, TransformerMixin ):
    #Class Constructor 
    def __init__( self ):
        self.scalers = {}
        for j in range(22):
            self.scalers[j] = StandardScaler()
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        for j in range(22):
            self.scalers[j].fit(X[:,0 ,j, :])
        return self 
    
    #Method that describes what we need this transformer to do
    def transform( self, X, y = None ):
        for j in range(22):
            X[:,0,j,:] = self.scalers[j].transform(X[:,0 ,j, :])
        return X

/home/thoriri/miniconda3/envs/EEG-TCNet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/thoriri/miniconda3/envs/EEG-TCNet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/thoriri/miniconda3/envs/EEG-TCNet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/thoriri/miniconda3/envs/E

# Making pipelines for EEG-TCNet

In [2]:
stand = [True,True,True,True,True,True,True,True,True]
for i in range(9):
    if not(os.path.exists('models/EEG-TCNet/S{:}/pipeline_fixed.h5'.format(i+1))):
        print('Making Pipeline for Subject {:}'.format(i+1))
        path_for_model = 'models/EEG-TCNet/S{:}/model_fixed.h5'.format(i+1)
        clf = KerasClassifier(build_fn = build_model, path = path_for_model)
        if(stand[i]):
            pipe = make_pipeline(Scaler(),clf)
        else:
            pipe = make_pipeline(clf)
        data_path = 'data/' 
        path = data_path+'s{:}/'.format(i+1)
        X_train,_,y_train_onehot,X_test,_,y_test_onehot = prepare_features(path,i,False)
        pipe.fit(X_train,y_train_onehot)
        X_train,_,y_train_onehot,X_test,_,y_test_onehot = prepare_features(path,i,False)
        y_pred = pipe.predict(X_test)
        dump(pipe, 'models/EEG-TCNet/S{:}/pipeline_fixed.h5'.format(i+1))
    else:
        print('Pipeline already exists for Subject {:}'.format(i+1))
print('Done!')

Pipeline already exists for Subject 1
Pipeline already exists for Subject 2
Pipeline already exists for Subject 3
Pipeline already exists for Subject 4
Pipeline already exists for Subject 5
Pipeline already exists for Subject 6
Pipeline already exists for Subject 7
Pipeline already exists for Subject 8
Pipeline already exists for Subject 9
Done!


# Making pipelines for Variable EEG-TCNet

In [3]:
stand = [True,False,True,True,True,True,True,True,True]
for i in range(9):
    if not(os.path.exists('models/EEG-TCNet/S{:}/pipeline.h5'.format(i+1))):
        print('Making Pipeline for Subject {:}'.format(i+1))
        path_for_model = 'models/EEG-TCNet/S{:}/model.h5'.format(i+1)
        clf = KerasClassifier(build_fn = build_model,path=path_for_model)
        if(stand[i]):
            pipe = make_pipeline(Scaler(),clf)
        else:
            pipe = make_pipeline(clf)
        data_path = 'data/' 
        path = data_path+'s{:}/'.format(i+1)
        X_train,_,y_train_onehot,X_test,_,y_test_onehot = prepare_features(path,i,False)
        pipe.fit(X_train,y_train_onehot)
        X_train,_,y_train_onehot,X_test,_,y_test_onehot = prepare_features(path,i,False)
        y_pred = pipe.predict(X_test)
        dump(pipe, 'models/EEG-TCNet/S{:}/pipeline.h5'.format(i+1))
    else:
        print('Pipeline already exists for Subject {:}'.format(i+1))
print('Done!')

Pipeline already exists for Subject 1
Pipeline already exists for Subject 2
Pipeline already exists for Subject 3
Pipeline already exists for Subject 4
Pipeline already exists for Subject 5
Pipeline already exists for Subject 6
Pipeline already exists for Subject 7
Pipeline already exists for Subject 8
Pipeline already exists for Subject 9
Done!


# Accuracy and Kappa score calculation for EEG-TCNet

In [4]:
for i in range(9):
    clf = load('models/EEG-TCNet/S{:}/pipeline_fixed.h5'.format(i+1)) 
    data_path = 'data/'
    path = data_path+'s{:}/'.format(i+1)
    X_train,_,y_train_onehot,X_test,_,y_test_onehot = prepare_features(path,i,False)
    y_pred = clf.predict(X_test)
    acc_score = accuracy_score(y_pred,np.argmax(y_test_onehot,axis=1))
    kappa_score = cohen_kappa_score(y_pred,np.argmax(y_test_onehot,axis=1))
    print('For Subject: {:}, Accuracy: {:}, Kappa: {:}.'.format(i+1,acc_score*100, kappa_score))








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

For Subject: 1, Accuracy: 85.76512455516014, Kappa: 0.8101928467695633.
For Subject: 2, Accuracy: 65.01766784452296, Kappa: 0.5339432753888381.
For Subject: 3, Accuracy: 94.5054945054945, Kappa: 0.926729767932867.
For Subject: 4, Accuracy: 64.91228070175438, Kappa: 0.5318755774561132.
For Subject: 5, Accuracy: 75.36231884057972, Kappa: 0.671779087459121.
For Subject: 6, Accuracy: 61.395348837209305, Kappa: 0.4850076476869354.
For Subject: 7, Accuracy: 87.36462093862815, Kappa: 0.8317628889235948.
For Subject: 8, Accuracy: 83.76383763837639, Kappa: 0.7835188177411448.
For Subject: 9, Accuracy: 78.03030303030303, Kappa: 0.7066441872940455.


# Accuracy and Kappa score calculation for Variable EEG-TCNet

In [5]:
for i in range(9):
    clf = load('models/EEG-TCNet/S{:}/pipeline.h5'.format(i+1)) 
    data_path = 'data/'
    path = data_path+'s{:}/'.format(i+1)
    X_train,_,y_train_onehot,X_test,_,y_test_onehot = prepare_features(path,i,False)
    y_pred = clf.predict(X_test)
    acc_score = accuracy_score(y_pred,np.argmax(y_test_onehot,axis=1))
    kappa_score = cohen_kappa_score(y_pred,np.argmax(y_test_onehot,axis=1))
    print('For Subject: {:}, Accuracy: {:}, Kappa: {:}.'.format(i+1,acc_score*100, kappa_score))

For Subject: 1, Accuracy: 89.32384341637011, Kappa: 0.8576302100925488.
For Subject: 2, Accuracy: 72.43816254416961, Kappa: 0.6325715331990611.
For Subject: 3, Accuracy: 97.43589743589743, Kappa: 0.9658072250353379.
For Subject: 4, Accuracy: 75.87719298245614, Kappa: 0.6782800554158757.
For Subject: 5, Accuracy: 83.69565217391305, Kappa: 0.7826885727783319.
For Subject: 6, Accuracy: 70.69767441860465, Kappa: 0.6094853683148335.
For Subject: 7, Accuracy: 93.14079422382672, Kappa: 0.9085903848825899.
For Subject: 8, Accuracy: 86.71586715867159, Kappa: 0.822837219437786.
For Subject: 9, Accuracy: 85.22727272727273, Kappa: 0.8029247377689304.
